# Lec 1

## Objective

General setup:

- Install dependencies
- Import TensorFlow Layers
- Config GPU
- Create Data folder

## 1.1 Install and import dependencies

First create a virtual environment for the project

In [2]:
#%python3 -m venv venv

# Activate the virtual environment
#%source venv/bin/activate

Then install the dependencies

In [3]:
#%pip install tensorflow 

In [4]:
#%pip install opencv-python

In [5]:
#%pip install matplotlib

Now import some necessary dependencies

In [6]:
import cv2 # OpenCV
import os  # For file operations
import random
import numpy as np
from matplotlib import pyplot as plt # For plotting graphs

In [ ]:
# Import TensorFlow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPooling2D,  Dense, MaxPool2D, Flatten, Input
import tensorflow as tf


- Conv2D for Convolutional Neural Networks
- Dense for Fully Connected Neural Networks
- MaxPooling2D to pull our layers together and effectively reduce the number of parameters and computations in the network
- Flatten take output from previous layer and flatten it to a vector to be fed into the next layer

## 1.2 Setup GPU Limit for TensorFlow

In [8]:
# Avoid Memory Error when TensorFlow use two much VRAM
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
gpus # Show all GPUs available in machine

## 1.3 Create Data Folder

When we perform facial verification, we actually want to compare two images and determine if they are of the same person. This is a one-shot learning problem. We will learn a function that maps our input data (images) to a feature space such that the distance between the features can be used to determine if the images are of the same person.

We will create three data folders:

- anchors
- positive
- negative

The anchor image is the image of the person we want to verify.(Input)
The positive or negative are verification images. The positive is an image of the same person as the anchor image but taken under different conditions. The negative image is an image of a different person.

When we pass in a anchor image, if:

- it matches the positive image, the output should be 1 (same)
- it matches the negative image, the output should be 0 (different)

We want our model to distinguish between outselves and others person. The process will be as follows:


Fix spelling later :)))

![AnchorAndPositive](resources/images/AnchorAndPositive.jpg)

![AnchorAndPNegative](resources/images/AnchorAndNegative.jpg)

**Labeled Face Wild Dataset (LFW)** is a dataset for studying Face Verification. It contains 13,000 images of faces collected from the web. Each face has been labeled with the name of the person pictured. We use this date set as negative images to train the model distinguishing outselves from others. We will explore this dataset in the lecture 2.

In [10]:
#Set up Paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

# Why need 'join' ? -> Because it will automatically add the correct path separator for the OS like '/' or '\'

In [ ]:
POS_PATH

In [12]:
# Create those directories in machine
os.makedirs(POS_PATH, exist_ok=True) # DO not overwrite if already exists
os.makedirs(NEG_PATH, exist_ok=True)
os.makedirs(ANC_PATH, exist_ok=True)

# Lec 2

## Objective

- Unpack Labeled Faced in the Wild (LFW) dataset
- Collect Anchr images
- Collect Positive images

## 2.1 Unpack LFW dataset for Negative Images

Found in this link: https://vis-www.cs.umass.edu/lfw/

Download -> All images as gzipped tar file -> lfw.tgz. Then move to the project workspace and extract the file using:

```bash
tar -xvzf lfw.tgz
```


![datasetView1](resources/images/datasetView1.png)


After extract, we see the structure of the dataset contains many folders, each folder is a person's name and contains some image(s) of that person inside the folder. We now want to collect all these images in the wholde dataset and put them into our `data/negative` folder. (Only need to take the images, not the folders outside).

**Uncomment the above code when run the notebook for the first time**

In [13]:
# # Define the path to the extracted LFW dataset
# LFW_PATH = os.path.join('lfw')
# #

# #  Collect all images from the LFW dataset and move them to the 'data/negative' folder
# for dirpath, dirnames, filenames in os.walk(LFW_PATH):
#     for filename in filenames:
#         if filename.endswith('.jpg'):
#             src_path = os.path.join(dirpath, filename)
#             dst_path = os.path.join(NEG_PATH, filename)
#             os.replace(src_path, dst_path)

It should look something like this in out `data/negative` folder:

![datasetView2](resources/images/datasetView2.png)

## 2.2 Collect Positive and Anchor Images

We will collect these images **from our webcam** using OpenCV. The images from LFW have size 250x250, so we will resize the images from our webcam to 250x250 as well.

**First, test the webcam:**


In [14]:
# Defined the Camera ID to use
CAM_ID = 3 # Establishing the connection with the IR camera


- CAM_ID = 0 for laptop normal webcam
- CAM_ID = 2 for laptop IR webcam
- CAM_ID = 4 for external webcam


***Depend on each devices, these number can be different. Try out all number start from 0 and see which one is the correct one on your device.***


Set up camera:

**IR webcam:**

What is IR webcam? https://fptshop.com.vn/tin-tuc/danh-gia/ir-camera-la-gi-153147


**External webcam**

Since the resolution of the laptop webcam is not good, we will use an external webcam like from a mobile phone. To connect the webcam to the laptop:

1. Download the DroidCam app on your phone, also the DroidCam client on your laptop: https://www.dev47apps.com/
2. Set up as the instruction on the website. With Linux:

```bash
wget -O droidcam_latest.zip https://files.dev47apps.net/linux/droidcam_2.1.3.zip
unzip droidcam_latest.zip -d droidcam
cd droidcam && sudo ./install-client
sudo apt install libappindicator3-1

# Fix missing video device
sudo apt install linux-headers-`uname -r` gcc make
sudo ./install-video
```
3. Open both DroidCam on phone and DroidCam client on laptop, connect the phone to the laptop via USB or Wifi.


Now try to use OpenCV to capture the image from the webcam:


In [15]:
cap = cv2.VideoCapture(CAM_ID)  # Connect to the camera
while cap.isOpened():  # Loop through every single frame
    ret, frame = cap.read()  # Read the frame
    if not ret:
        break

    cv2.imshow('Image Collection', frame)  # Display the frame to screen

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Wait for 1ms and check if 'q' is pressed
        break

cap.release() 
cv2.destroyAllWindows()  # Close the window

**Press `q` to exit**


After that, we show to last frame/image captured when press `q`:

In [ ]:
plt.imshow(frame) # Display the image

Note that by default, OpenCV reads images in **BGR** (Blue, Green, Red) format so the image will look a bit weird.

In [ ]:
frame.shape # Check the shape of the image

In [ ]:
frame

When using normal webcam, the result look something like this:

```plaintext
array([[[181, 163, 164],
        [186, 168, 168],
        [186, 164, 168],
        ...,
        [104, 107, 104],
        [104, 106, 106],
        [105, 107, 107]],

       [[183, 167, 167],
        [183, 167, 167],
        [180, 162, 163],
```
Each [181, 163, 164] is a pixel value of the image. The first value is the Blue channel, the second is the Green channel, and the third is the Red channel. The value of each channel is from 0 to 255.

Whereas when using IR webcam, the result look something like this:

```plaintext
array([[[16, 16, 16],
        [15, 15, 15],
        [23, 23, 23],
        ...,
        [ 6,  6,  6],
        [ 7,  7,  7],
        [ 6,  6,  6]],
```

Each picxel is just a single value, which is the gray intensity of the pixel. The value is from 0 to 255. However they are write duplicated 3 times to match the RGB format, so when use frame.shape, it will return (480, 640, 3) instead of (480, 640, 1).

However, this is not 250x250 yet, we will resize the image to 250x250.




In [ ]:
#Resize the image to 250x250
resized_frame = cv2.resize(frame, (250, 250))
plt.imshow(resized_frame)

### Face Detection

Just resize the image to 250x250 will make the image look weird, so instead, we should crop the image to 250x250, by not by arbitrary, but by the face of the person in the image. We will use the Haar Cascade Classifier to detect the face in the image and crop the image to the face.

In [ ]:
# Load the Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Convert the frame to grayscale as the face detector expects gray images
gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)

# If faces are detected, crop the first face found
if len(faces) > 0:
    (x, y, w, h) = faces[0]  # Get the coordinates of the first face
    cropped_face = frame[y:y+h, x:x+w]  # Crop the face from the frame

    # Resize the cropped face to 250x250
    resized_face = cv2.resize(cropped_face, (250, 250))

    # Display the resized face
    plt.imshow(cv2.cvtColor(resized_face, cv2.COLOR_BGR2RGB))
    plt.show()
else:
    print("No faces detected")

### Collect images to anchor and positive folders

Now we will collect the images to the anchor and positive folders. Create a capture frame, then when press `a`, detect the face in current frame then crop that face to 250x250 image then save to the anchor folder. When press `p`, the image will be saved to the positive folder. notify to the user when the image is saved successfully or not

In [21]:
import uuid # For generating unique image file names

# Function to save the captured image to the specified folder
def save_image(image, folder_path, img_name):
    img_path = os.path.join(folder_path, img_name)
    cv2.imwrite(img_path, image)

# Initialize the webcam
cap = cv2.VideoCapture(CAM_ID)
# Load the Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Loop through every frame in the webcam feed
while cap.isOpened():
    
    ret, frame = cap.read()
    if not ret:
        break



    # Display the frame
    cv2.imshow('Face Collection', frame)

    # Check for key presses
    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        # Detect face in the frame then crop to 250x250 around the face and save to 'data/anchor'
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            cropped_face = frame[y:y+h, x:x+w]
            resized_face = cv2.resize(cropped_face, (250, 250))
            save_image(resized_face, ANC_PATH, str(uuid.uuid1())+ ".jpg")
            print("Image saved as anchor")
        else:
            # Show a dialog if no faces are detected
            print("No faces detected")


    elif key == ord('p'):
        # Detect face in the frame then crop to 250x250 around the face and save to 'data/positive'
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
        if len(faces) > 0:
            (x, y, w, h) = faces[0]
            cropped_face = frame[y:y+h, x:x+w]
            resized_face = cv2.resize(cropped_face, (250, 250))
            save_image(resized_face, POS_PATH, str(uuid.uuid1())+ ".jpg")
            print("Image saved as positive")
        else:
            # Show a dialog if no faces are detectedq
            print("No faces detected")

    elif key == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

Tim hieu them ve tuat toanFace Detection nay ????

# Lec 3

## Objective

- Preprocess the images
- Create positive and negative samples
- Load data into TensorFlow Dataloader

## 3.1 Get Image Directories

In [22]:
# Load directories to TensorFlow datasets
# Take 50 images from each folder
anchor = tf.data.Dataset.list_files(os.path.join(ANC_PATH, '*.jpg')).take(50) # This create a pipeline of all images in the folder
positive = tf.data.Dataset.list_files(os.path.join(POS_PATH, '*.jpg')).take(50)
negative = tf.data.Dataset.list_files(os.path.join(NEG_PATH, '*.jpg')).take(50)

We take an example path from `anchor` folder for testing purpose in Preprocess step.

In [ ]:
# Take an example path from the anchor dataset
example_anchor_path = next(iter(anchor))

# Print the example path
print(example_anchor_path.numpy())

## 3.2 Preprocess the images

In [24]:
def preprocess(file_path):
    # Read the image from the file path
    byte_image = tf.io.read_file(file_path) # Read the image as bytes
    image = tf.image.decode_jpeg(byte_image) # Decode the image from bytes to tensor

    # Preprocess the image
    image = tf.image.resize(image, (100,100)) # Resize the image to 100x100
    image = image / 255.0 # Scale image pixel values range [0,1]
    return image

The correct dimension to resize will base on the model we use. For example:

- VGGFace: 224x224
- Facenet: 160x160
-....

Thu tim them cac model khac va kich thuoc anh phu hop


Test the preprocess function with the example path from the previous step.

In [ ]:
# Images before preprocessing
plt.imshow(cv2.imread(example_anchor_path.numpy().decode()))
# Value range
print("Value range before preprocessing:", frame.min(), "-", frame.max())
plt.show()


# Images after preprocessing
preprocessed_image_test = preprocess(example_anchor_path)
print("Value range before preprocessing:", preprocessed_image_test.numpy().min(), "-", preprocessed_image_test.numpy().max())
plt.imshow(preprocessed_image_test)
plt.show()

## 3.3 Create labled Dataset

We want to create a dataset that labels as follow:

- Given two input: Anchor + Positive => 1
- Given two input: Anchor + Negative => 0

In [26]:
# Create labels
positive_labels = tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor))) #each label has the same size as the anchor
negative_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))

# Zip the datasets with their labels
positives = tf.data.Dataset.zip((anchor, positive, positive_labels))
negatives = tf.data.Dataset.zip((anchor, negative, negative_labels))

# Concatenate the datasets
data = positives.concatenate(negatives)

In [ ]:
# View the first 5 examples
# Print the dataset to verify
for element in data.take(5):
    print(element)

In [ ]:
# Example of a sample from the dataset
samples = data.as_numpy_iterator()
samples.next()

## 3.4 Build Train and Test Parition

Now we apply the `preprocess` function to each sample in the dataset. Curretly, the `preprocess` function only precess one passed image, so we create a new function to process two images at the same time (just calling the `preprocess` function twice inside that).

In [29]:
def preprocess_twin(input_img, validation_img, label):
    input_img = preprocess(input_img)
    valid_img = preprocess(validation_img)
    return input_img, valid_img, label

# input_img will be the anchor image, valid_img will be the positive or negative image

In [ ]:
# Test with a sample
samples = data.as_numpy_iterator()
sample = samples.next()

res = preprocess_twin(*sample) # `*` is used to unpack the tuple
res

Output the result in image format to see the result.

In [ ]:
print("Anchor image: ")
plt.imshow(res[0])
plt.show()

print("Validation image: ")
plt.imshow(res[1])
plt.show()

# Directly print the value of res[2]
print("Label:", res[2])

After tesing all good for a single sample, we apply this process to all samples in the dataset.

In [32]:
data = data.map(preprocess_twin)
data = data.cache() # Cache the dataset to memory to get a speedup while reading from it

Currently, the dataset contains **positive classes consecutively, then negative classes consecutively**. We need to shuffle the dataset to make it more random.

In [ ]:
# Before mix
for element in data.take(10):
    print(element[2].numpy())  # Print the label to see the mix of classes
# See if the appearance of 0 and 1 is random yet

In [34]:
# Shuffle the dataset so that we have a good mix of positive and negative examples
data = data.shuffle(buffer_size=1024) 

In [ ]:
# Check the dataset again to see that the two classes are mixed well
for element in data.take(10):
    print(element[2].numpy())  # Print the label to see the mix of classes
# See if the appearance of 0 and 1 is random yet

Now we split the Dataset in to Training and Testing set. We will use **70% for training and 30% for testing**.

In [36]:
train_data = data.take(round(len(data)*0.7)) # 70% of the data for training
train_data = train_data.batch(16) # Batch size of 16; batch size" refers to the number of training examples utilized in one iteration of the training process.
train_data = train_data.prefetch(8) # start processing the next batch while the current batch is being processed

In [ ]:
print("Size of original dataset:", len(data))
print("Size of training dataset:", len(list(train_data.as_numpy_iterator())))

In [ ]:
# Note that now each sample in train_data is 16 images
for element in train_data.take(1):
    print(f"Size of each sample in train_data: {element[0].shape}, {element[1].shape}, {element[2].shape}")

(16, 100, 100, 3) represents the anchor images in one single sample:

- 16: meaning there are 16 images in this batch.
- 100, 100: the height and width of each image, indicating that each image is 100 pixels by 100 pixels.
- 3: the number of color channels in each image. We store the image as Gray scale, but the vluae is duplicated 3 times to match the RGB format.

Similiarly for the remaining two elements in the tuple.

In [ ]:
# Print the shape and values of element[0][0] (the first image in the first batch) to verify if it is stored as a grayscale image
print("Shape of element[0][0]:", element[0][0].shape)
print("Values of element[0][0]:")
print(element[0][0].numpy())

plt.imshow(element[0][0]) # Display the first image in the batch

In [40]:
# Do similar for the test data 30%
test_data = data.skip(round(len(data)*0.7)) # Skip the first 70% of the data which is used for training
test_data = test_data.take(round(len(data)*0.3)) 
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [ ]:
print("Size of original dataset:", len(data))
print("Size of training dataset:", len(list(train_data.as_numpy_iterator())))
print("Size of test dataset:", len(list(test_data.as_numpy_iterator())))

# Lec 4

## Objective

- Build an embedding layer / Encode Layer
- Create an L1 Distance layer
- Complie the Siamase Network

## 4.1 Build an Embedding Layer

Recall th idea, we have two stream of information (we  pass in two images: the anchor and the positive/negative image) -> Each stream will pass through an embedding layer to get the feature vector of the image -> Tow feature vectors will be used to calculate the distance between the two images at the Distance layer (here we use L1 distance).


Here is the Siamese Network architecture we are going to build:

![SiameseStructure](resources/images/SiameseStructure.png)

In the paper, it use input size of 105x105, but we will use 100x100 still okey. Therefore, **some numbers may not match exactly to the paper**

In [143]:
def make_embedding() :

    #Create the input layer
    inp = Input(shape=(100, 100, 3), name='input_image') # 100x100 image size, 3 channels color

   
    ### First Block ###
    # Next layer is a convolutional layer 64 filters, kernel size of 10x10, and ReLU activation
    c1= Conv2D(64, (10,10), activation='relu')(inp) # c1 stands for convolutional layer 1

    # Next layer is a max pooling layer with a pool size of 2x2
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1) 



    ### Second Block ###
    # Next layer is a convolutional layer 128 filters, kernel size of 7x7, and ReLU activation
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    # Next layer is a max pooling layer with a pool size of 2x2
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)



    ### Third Block ###
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)

    ### Fourth Block ###
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4) # Flatten the output of the convolutional layer to feed it to the dense layer
    d1 = Dense(4096, activation='sigmoid')(f1) # Dense layer with 4096 neurons and sigmoid activation

    
    return Model(inputs=inp, outputs=d1, name='Siamese__embedding_model')

As show in the above image, the output of the Embedding layer is a 4096-dimensional feature vector. Two streams of information will pass through this Embedding layer then we get two 4096-dimensional feature vectors for each image. **Basicaly, we are converting the face image to a 4096-dimensional feature vector.**

In [ ]:
# Summary of the model
siamese_embedding_model = make_embedding()
siamese_embedding_model.summary()

How to read from the summary table:

This is a 4-dimensional tensor shape where each dimension represents, for example (None,100,100,3):

- None: This represents the batch size dimension. It's set to "None" because it's flexible - you can feed any number of images through the network at once.
- 100: The height of your input images (100 pixels)
- 100: The width of your input images (100 pixels)
- 3: The number of color channels

So in plain terms, your model is expecting input images that are:

- 100x100 pixels in size
- Color in 3 channels, can be RGB, BRG or even just grayscale but duplicated 3 times to match the format.
- Can be processed in batches of any size (that's what the None indicates)

## 4.2 Create an L1 Distance Layer

Compare two feature vectors to see how similiar our two images are

In [166]:

class L1Dist(Layer):


    def __init__(self, **kwargs):
        super().__init__()
    
    def call(self, input_embedding, validation_embedding):

        # Convert inputs to tensors otherwise will meet error: un supported operand type(s) for -: 'List' and 'List'
        input_embedding = tf.convert_to_tensor(input_embedding)
        validation_embedding = tf.convert_to_tensor(validation_embedding)
        input_embedding = tf.squeeze(input_embedding, axis=0)  # Remove potential first dimension
        validation_embedding = tf.squeeze(validation_embedding, axis=0)

        # Calculate and return the L1 distance
        return tf.math.abs(input_embedding - validation_embedding)

    

## 4.3 Create final Siamese Network

We combine two above steps to make a fully Siamese model

In [174]:
def make_siamese_model():

    
    # Handle input images
    input_image = Input(shape=(100, 100, 3), name='input_image') # Anchor image
    validation_image = Input(shape=(100, 100, 3), name='validation_image') # Positive or negative image


    # Pass through the embedding model/Encoder layer to get the features vector
    embedding = make_embedding()
    input_embedding = embedding(input_image)
    validation_embedding = embedding(validation_image)


    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(input_embedding, validation_embedding)
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='Fully_Siamese_Network')


In [ ]:
# Create the model
fully_siamese_model = make_siamese_model()
fully_siamese_model.summary()

Look at the summary table, we see that the input of the model is two images size 100,100,3. (How many images per batch is not specified, so it's flexible). The output is a **single value**, which is the distance between the two images.

# 5. Train the Siamese Network

## Objective

- Set up a loss function, set up an optimizer
- Establish checkpoints
- Build a Custom Training Step
- Create a Training Loop
- Train the model

## 5.1 Set up a loss function and an optimizer

In [176]:
#Define the loss function
binary_cross_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# from_logits=True is recommened when thr inputs to the loss function are not normalized 

In [177]:
# Define the optimizer
optimizer = tf.keras.optimizers.Adam(1e-4)

There are tons of optimizer for Keras: SGD, Adam, RMSprop, Adagrad, Adadelta, Adamax, Nadam, Ftrl. We will use Adam optimizer.

## 5.2 Establish checkpoints

This checkpoint will save the model after each epoch. In case the training process is interrupted, we can resume the training from the last checkpoint.

In [178]:
# Create training_checkpoints directory in th machine
os.makedirs('./training_checkpoints', exist_ok=True)


checkpoint_dir = './training_checkpoints' # Directory to save the checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt") #Prefix for the checkpoint files with unique number
checkpoint = tf.train.Checkpoint(optimizer=optimizer, siamese_model=fully_siamese_model)

# To reload the model from the checkpoint, use model.load('path_to_checkpoint'). This will
# load pre trainweights and optimizer state into existing model

## 5.3 Build a Custom Training Step

In this step, we defined actual trainning steps. We train on one batch of data, one batch of data come through our training step, we go on makeing prediction -> calcualte our loss function -> calculate gradient then apply back popagation (calculate new weights and apply) through our neutral network to get the best possible model

In [182]:
@tf.function # Annotation to indicate that complies to TensorFlow graph execution 
def train_step(batch):

    with tf.GradientTape() as tape:  # Record operations for automatic differentiation
        
        # Get the input images, validation images and labels from a batch
        input_images, validation_images, labels = batch

        
        # Forward pass
        predictions = fully_siamese_model([input_images, validation_images], training=True)
        # Set training=True for is important since some layers will only activated when this is set to True
    
    
        # Calculate the loss
        loss = binary_cross_loss(labels, predictions) # Measure the difference between the predicted value and the actual value
        print("Loss Vaule: ", loss)
    
        # Calculate the gradients
        gradients = tape.gradient(loss, fully_siamese_model.trainable_variables)

        # Update the weights and apply to the Siamese model
        optimizer.apply_gradients(zip(gradients, fully_siamese_model.trainable_variables))
        # Adam is a variant of stochastic gradient descent, it applies te learning rate and gradient to slightly reduce the loss function, unitll
        # it realy near the minimum value.



Recall the batch we created before:


In [ ]:
# take one testing batch from train_data
test_batch = train_data.as_numpy_iterator()
test_batch = test_batch.next()
print("The length of the batch is", len(test_batch))
print("The first element of the batch is 16 anchor images: ", test_batch[0].shape)
print("The second element of the batch is 16 validation images: ", test_batch[1].shape)
print("The third element of the batch is a single label 1 or 0 (but write duplicated 16 times): ", test_batch[2].shape)

## 5.4 Build Training Loop

In [183]:
def train(data, EPOCHS): # EPOCHS is the number of times the model will see the entire dataset

    # Loop through the epochs
    for epoch in range(1,EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS)) # Print the current epoch to know the progress of runnig
        progressBar = tf.keras.utils.Progbar(len(data))

        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run the training step here
            train_step(batch)
            progressBar.update(idx+1) # Update the progress bar

    # Save our checkpoint 
    if (epoch) % 10 == 0: # Save the model every 10 epochs
        checkpoint.save(file_prefix = checkpoint_prefix)

## 5.5 Train the model

In [ ]:
# Define the number of epochs
EPOCHS = 20

train(train_data, EPOCHS)